In [ ]:
import torch
import os
from google.colab import files
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

In [ ]:
# 1. GPU 사용 가능 여부 확인 및 device 설정

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("사용 가능한 device:", device)
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

사용 가능한 device: cuda


In [ ]:
# 2. 내 PC의 엑셀 파일 업로드 (Colab에서 파일 업로드 창이 뜹니다)
uploaded = files.upload()  # 여기서 'spam_data.xlsx' 등 파일을 선택하세요

Saving lgaidataset.csv to lgaidataset (1).csv


In [ ]:
# 3. 업로드한 CSV 파일을 pandas DataFrame으로 읽어오기

# CSV 파일 이름은 업로드한 파일명과 동일하게 사용하세요.
data = pd.read_csv('lgaidataset.csv')  # pd.read_excel 대신 pd.read_csv 사용
print("데이터 미리보기:")
print(data.head())

데이터 미리보기:
   index                                            content  class
0    1.0  엄마 나 폰이 망가져서 수리맡기고 컴퓨터 문자나라로 메시지 보내고 있어 엄마 지금 바뻐?      1
1    2.0  [국외발신]\n[코인원]\n고객님계정이\n해외IP에서\n로그인되였습니다.해외IP차단...      1
2    3.0  엄마 입금받을수있는\n은행계좌번호\n하나랑 계좌등록\n본인인증땜에\n계좌 4자리\n...      1
3    4.0  엄마 바빠?나지금\n핸드폰 고장나서 매장에 수리맡기고\n급한대로 예전에\n내명의로 ...      1
4    5.0  엄마~ 바빠?\n엄마 나 급한 일이 좀 생겨서…\n지금 98만원만 입금해 줄 수 있...      1


In [ ]:
data['class'].unique()

array([1, 2, 0, 3, 4])

In [ ]:
# data = data.dropna(subset=['index'])

In [ ]:
# 1. index 열에서 현재 가장 높은 값 찾기
max_index = data['index'].max()  # 예: 33436.0

# 2. NaN 값의 개수 파악
nan_count = data['index'].isna().sum()  # 예: 1

# 3. NaN을 채울 새로운 값 생성 (최대값 + 1부터 순차적으로)
new_values = range(int(max_index) + 1, int(max_index) + 1 + nan_count)  # 예: [33437]

# 4. NaN이 있는 위치에 새로운 값 할당
data.loc[data['index'].isna(), 'index'] = new_values

In [ ]:
data.loc[:, 'class'] = data['class'] + 1

In [ ]:
data['class'].unique()

array([2, 3, 1, 4, 5])

In [ ]:
# 4. Train, Test 데이터 분할 (전체 데이터의 20%를 테스트셋으로 사용)

train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)
print("Train 데이터 크기:", train_df.shape)
print("Test 데이터 크기:", test_df.shape)

Train 데이터 크기: (43060, 3)
Test 데이터 크기: (10765, 3)


In [ ]:
train_df.head(5)

,index,content,class
15814,17815.0,세월호랑 비교하다니....,1
51057,17768.0,"ⓙⓙⓙ***.com서울-cⓐsino갸 ***,***입 지굽",4
53274,45359.0,[web발신]\n[쿠팡] 로캣배송 노트북 문 앞(으)로 배송 완료했습니다.\n사진:...,5
7184,9185.0,어떻게 감당할지 여전히 어렵고 모르겠어요.,1
31435,33436.0,그럼 안녕히계세요^^,1


In [ ]:
print(train_df['class'].unique())
print(test_df['class'].unique())
print(train_df['content'].head())
print(test_df['content'].head())

[1 4 5 2 3]
[1 4 5 2 3]
15814                                       세월호랑 비교하다니....
51057                     ⓙⓙⓙ***.com서울-cⓐsino갸 ***,***입 지굽
53274    [web발신]\n[쿠팡] 로캣배송 노트북 문 앞(으)로 배송 완료했습니다.\n사진:...
7184                               어떻게 감당할지 여전히 어렵고 모르겠어요.
31435                                          그럼 안녕히계세요^^
Name: content, dtype: object
32973                  안희정의 부드러운 강인함과 인간적 매력이 승리하지 않을까 싶다.
24888                                        헤어진지 2일정도 됐어요
31689                       이것이 참 대한민국이다~! Go Korea ~~~!!!
19855                                계속 풀어도 계속 나옴 잠 못 잠 ㅇㅇ
1221     옆 옆 옆집 과일주스 가게 왔는데 이모님께서 나 평소에 많이 예뻐해 주셔 가지고 언...
Name: content, dtype: object


In [ ]:
# 5. 텍스트 전처리 및 토큰화 함수 정의

nltk.download('punkt')  # 처음 한 번만 실행하면 됩니다
nltk.download('punkt_tab')

def preprocess_text(text):
    """
    입력된 텍스트를 소문자화, 특수문자 제거 후 단어 단위로 토큰화합니다.
    """
    text = text.lower()  # 모두 소문자로 변환
    text = re.sub(r'[^가-힣a-z0-9\s]', '', text)
    tokens = word_tokenize(text)  # 단어 단위 토큰화
    return tokens

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


아무 데이터에 관해 preprocess_text 함수 실행해서 한국어 토큰화 되는지 확인하기

In [ ]:
# 6. Vocabulary(단어 집합) 구축

all_tokens = []
# train 데이터의 모든 메시지에 대해 토큰을 추출합니다.
for msg in train_df['content']:
    tokens = preprocess_text(msg)
    all_tokens.extend(tokens)

# 각 단어의 빈도수를 계산합니다.
word_counts = Counter(all_tokens)
# 자주 등장하는 단어만 사용 (여기서는 빈도수 1 이상인 단어 모두 사용)
# 인덱스 0은 <PAD>용, 1은 <UNK>(알 수 없는 단어) 용으로 예약합니다.
vocab = {word: i+2 for i, (word, count) in enumerate(word_counts.items()) if count >= 1}
vocab["<PAD>"] = 0
vocab["<UNK>"] = 1
vocab_size = len(vocab)+1
print("단어 집합 크기:", vocab_size)

단어 집합 크기: 121106


In [ ]:
# 7. 텍스트를 숫자 시퀀스로 변환하는 함수 (최대 길이 max_len으로 패딩 또는 자르기)
def text_to_sequence(text, vocab, max_len=50):
    """
    텍스트를 토큰화한 후 단어를 해당 인덱스로 변환합니다.
    max_len보다 짧으면 패딩(<PAD>), 길면 자릅니다.
    """
    tokens = preprocess_text(text)
    seq = [vocab.get(token, vocab["<UNK>"]) for token in tokens]  # 단어가 없으면 <UNK> 사용
    if len(seq) < max_len:
        seq = seq + [vocab["<PAD>"]] * (max_len - len(seq))  # 부족한 길이만큼 패딩
    else:
        seq = seq[:max_len]  # max_len까지만 사용
    return seq

In [ ]:
# 8. PyTorch Dataset 클래스 정의 (메시지와 라벨을 숫자 시퀀스로 변환)

class SpamDataset(Dataset):
    def __init__(self, df, vocab, max_len=50):
        self.messages = df['content'].tolist()  # 메시지 리스트
        self.labels = df['class'].tolist()        # 라벨 리스트 (1~5)
        self.vocab = vocab
        self.max_len = max_len

    def __len__(self):
        return len(self.messages)

    def __getitem__(self, idx):
        message = self.messages[idx]
        label = self.labels[idx]
        # 메시지를 숫자 시퀀스로 변환
        seq = text_to_sequence(message, self.vocab, self.max_len)
        # 라벨은 0부터 시작하도록 변환 (예: 1→0, 2→1, …)
        return torch.tensor(seq, dtype=torch.long), torch.tensor(label-1, dtype=torch.long)

In [ ]:
# 9. DataLoader 생성 (배치 처리 및 셔플)
batch_size = 64
train_dataset = SpamDataset(train_df, vocab)
test_dataset = SpamDataset(test_df, vocab)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# 10. LSTM 기반 분류 모델 정의 (임베딩, LSTM, 선형 계층 포함)
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim, num_layers=1):
        super(LSTMClassifier, self).__init__()
        # 임베딩 레이어: 단어 인덱스를 임베딩 벡터로 변환 (padding_idx=0은 <PAD>를 위한 설정)
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        # LSTM 레이어: 시퀀스 데이터를 처리 (batch_first=True로 배치 차원이 첫번째)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers, batch_first=True)
        # 분류를 위한 선형(fully-connected) 레이어
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # x: [batch_size, seq_len]
        x = self.embedding(x)          # [batch_size, seq_len, embed_dim]
        out, (hn, cn) = self.lstm(x)     # LSTM 처리, out: 모든 타임스텝 출력
        out = out[:, -1, :]            # 마지막 타임스텝의 출력만 사용
        out = self.fc(out)             # 선형 계층 통과 → [batch_size, output_dim]
        return out

In [ ]:
# 모델 하이퍼파라미터 설정
embed_dim = 100    # 임베딩 차원
hidden_dim = 128   # LSTM hidden state 차원
output_dim = 5     # 분류할 클래스 개수 (1~5)
num_layers = 1
#dropout = 0.2     # 0.2 ~ 0.5

# 모델 초기화 및 device(GPU 또는 CPU)로 이동
model = LSTMClassifier(vocab_size, embed_dim, hidden_dim, output_dim, num_layers)
model.to(device)

# 11. 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()  # CrossEntropyLoss는 softmax 포함 (라벨은 0~4)
#optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0005)

In [ ]:
# 12. 학습 루프 실행
num_epochs = 3  # 에폭 수는 필요에 따라 조정 가능

for epoch in range(num_epochs):
    model.train()  # 학습 모드로 전환
    epoch_loss = 0
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()         # 기울기 초기화
        outputs = model(inputs)         # 모델 예측
        loss = criterion(outputs, labels)  # 손실 계산
        loss.backward()               # 역전파 수행
        optimizer.step()              # 가중치 업데이트

        epoch_loss += loss.item()

        if (batch_idx + 1) % 100 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Iteration {batch_idx+1}/{len(train_loader)}, Loss: {loss.item():.4f}")


    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(train_loader):.4f}")

Epoch 1/3, Iteration 100/673, Loss: 0.5748
Epoch 1/3, Iteration 200/673, Loss: 1.0266
Epoch 1/3, Iteration 300/673, Loss: 0.6004
Epoch 1/3, Iteration 400/673, Loss: 0.3866
Epoch 1/3, Iteration 500/673, Loss: 0.4577
Epoch 1/3, Iteration 600/673, Loss: 0.5886
Epoch 1/3, Loss: 0.6415
Epoch 2/3, Iteration 100/673, Loss: 0.6919
Epoch 2/3, Iteration 200/673, Loss: 0.3503
Epoch 2/3, Iteration 300/673, Loss: 0.5374
Epoch 2/3, Iteration 400/673, Loss: 0.4881
Epoch 2/3, Iteration 500/673, Loss: 0.5908
Epoch 2/3, Iteration 600/673, Loss: 0.4914
Epoch 2/3, Loss: 0.5043
Epoch 3/3, Iteration 100/673, Loss: 0.5417
Epoch 3/3, Iteration 200/673, Loss: 0.4692
Epoch 3/3, Iteration 300/673, Loss: 0.3929
Epoch 3/3, Iteration 400/673, Loss: 0.3599
Epoch 3/3, Iteration 500/673, Loss: 0.3158
Epoch 3/3, Iteration 600/673, Loss: 0.3594
Epoch 3/3, Loss: 0.3767


In [ ]:
# 13. 테스트 데이터로 모델 평가
model.eval()  # 평가 모드 전환 (dropout 등 비활성화)
correct = 0
total = 0

with torch.no_grad():  # 평가 시에는 기울기 계산 불필요
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)  # 가장 높은 확률의 클래스를 선택
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 75.42%


내가 넣은 입력을 분류하는 코드 추가

In [ ]:
def predict_message(text):
    """
    입력된 텍스트에 대해 전처리, 숫자 시퀀스 변환 후 모델 예측을 수행합니다.
    반환값은 예측된 클래스 번호(원래 라벨, 1~5)입니다.
    """
    model.eval()  # 평가 모드로 전환
    # 입력 텍스트를 숫자 시퀀스로 변환 (max_len 길이로 패딩 또는 자르기)
    seq = text_to_sequence(text, vocab, max_len=50)
    # 모델 입력으로 사용하기 위해 텐서로 변환하고 배치 차원 추가
    input_tensor = torch.tensor(seq, dtype=torch.long).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(input_tensor)
        # 가장 높은 확률을 가진 클래스를 선택 (0~4로 예측되었으므로, 실제 라벨은 +1)
        _, predicted = torch.max(output, 1)

    predicted_class = predicted.item() + 1
    return predicted_class

# 사용자로부터 직접 메시지 입력받기
sample_text = input("예측할 메시지를 입력하세요: ")
predicted_class = predict_message(sample_text)
print("예측된 클래스:", predicted_class)

예측할 메시지를 입력하세요: 엄마 나 폰이 잠겼는데 계좌이체좀 해줘
예측된 클래스: 4


Test Accuracy: 95.29%

🧠 전체 프로젝트 개요
우리가 만들고 있는 것은:

✉️ 스팸 메시지를 읽어서, 이게 1~5 중 어떤 종류의 스팸인지 분류하는 인공지능 모델입니다.

📦 1. GPU 사용 확인

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("사용 가능한 device:", device)
PyTorch에서 GPU를 사용할 수 있는지 확인합니다.

가능하면 cuda(GPU), 불가능하면 cpu로 설정합니다.

GPU를 쓰면 속도가 훨씬 빠릅니다.

📂 2. CSV 파일 업로드
from google.colab import files
uploaded = files.upload()
Colab은 내 컴퓨터에 있는 파일에 직접 접근 못 해요.

이 코드는 CSV 파일을 업로드할 수 있는 창을 띄워줍니다.

📊 3. CSV 읽기
import pandas as pd
data = pd.read_csv('spam_data.csv')
print("데이터 미리보기:")
print(data.head())
업로드한 CSV 파일을 판다스 데이터프레임으로 읽습니다.

data.head()는 상위 5개 행을 출력해서 데이터 구조를 확인합니다.

✅ 데이터에는 최소한 아래 두 컬럼이 있어야 해요:

message: 스팸 메시지 텍스트

label: 1 ~ 5 숫자로 된 클래스 정보

✂️ 4. 데이터 나누기 (Train/Test)
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)
데이터를 학습용(80%)과 테스트용(20%)으로 분리합니다.

모델이 안 본 데이터로 성능을 평가하려면 꼭 나눠야 합니다.

🔡 5. 텍스트 전처리 + 토큰화
import re
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
word_tokenize: 문장을 단어 단위로 잘라주는 함수

전처리 함수 만들기:

def preprocess_text(text):
    text = text.lower()  # 모두 소문자로
    text = re.sub(r'[^a-z0-9\s]', '', text)  # 특수문자 제거
    tokens = word_tokenize(text)  # 단어로 쪼갬
    return tokens
예: "WINNER! Claim your $100 now!"
→ ["winner", "claim", "your", "100", "now"]

📚 6. 단어 사전 (Vocabulary) 만들기
from collections import Counter

all_tokens = []
for msg in train_df['message']:
    tokens = preprocess_text(msg)
    all_tokens.extend(tokens)

word_counts = Counter(all_tokens)
vocab = {word: i+2 for i, (word, count) in enumerate(word_counts.items()) if count >= 1}
vocab["<PAD>"] = 0
vocab["<UNK>"] = 1
모든 메시지의 단어들을 모아 단어장(vocab)을 만듭니다.

<PAD>: 시퀀스를 고정된 길이로 만들기 위한 빈 자리

<UNK>: 모르는 단어

🔢 7. 텍스트를 숫자 시퀀스로 변환
def text_to_sequence(text, vocab, max_len=50):
    tokens = preprocess_text(text)
    seq = [vocab.get(token, vocab["<UNK>"]) for token in tokens]
    if len(seq) < max_len:
        seq += [vocab["<PAD>"]] * (max_len - len(seq))
    else:
        seq = seq[:max_len]
    return seq
단어들을 숫자 인덱스 리스트로 바꿉니다.

길이를 max_len에 맞게 패딩하거나 자릅니다.

🧱 8. PyTorch Dataset 만들기
from torch.utils.data import Dataset

class SpamDataset(Dataset):
    def __init__(self, df, vocab, max_len=50):
        self.messages = df['message'].tolist()
        self.labels = df['label'].tolist()
        self.vocab = vocab
        self.max_len = max_len
        
    def __len__(self):
        return len(self.messages)
    
    def __getitem__(self, idx):
        message = self.messages[idx]
        label = self.labels[idx]
        seq = text_to_sequence(message, self.vocab, self.max_len)
        return torch.tensor(seq, dtype=torch.long), torch.tensor(label - 1, dtype=torch.long)
PyTorch 모델이 데이터를 읽을 수 있도록 Dataset 클래스를 만듭니다.

__getitem__은 하나의 샘플을 숫자 시퀀스 + 정답(label)으로 반환합니다.

🚚 9. DataLoader로 배치 처리
from torch.utils.data import DataLoader

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
DataLoader는 데이터를 배치 단위로 묶어 모델에 공급해줍니다.

학습 데이터는 섞고(shuffle=True), 테스트는 그대로 유지합니다.

🧠 10. LSTM 모델 만들기
import torch.nn as nn

class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim, num_layers=1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        x = self.embedding(x)       # [batch, seq, embed_dim]
        out, (hn, cn) = self.lstm(x)
        out = out[:, -1, :]         # 마지막 시점의 hidden state
        return self.fc(out)         # [batch, output_dim]
embedding: 단어 → 벡터

lstm: 문장의 흐름(시퀀스)을 이해

fc: 마지막 출력 → 클래스 5개로 분류

⚙️ 11. 손실 함수 & 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
CrossEntropyLoss: 다중 클래스 분류용 손실 함수

Adam: 대표적인 옵티마이저

🔁 12. 모델 학습
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
학습 모드로 전환 (model.train())

데이터 → 모델 → 손실 계산 → 역전파 → 가중치 업데이트

이 과정을 여러 번 반복하며 모델이 학습됩니다.

📈 13. 모델 평가
model.eval()
correct = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs.to(device))
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels.to(device)).sum().item()
평가 모드 전환 (model.eval())

실제 정답과 예측을 비교해 정확도 계산

✅ 결과 출력
print(f"Test Accuracy: {100 * correct / total:.2f}%")
테스트 데이터에서 정확도가 몇 % 나오는지 출력합니다.

💬 마무리
이제 당신의 모델은:

CSV로 된 메시지를 읽고

텍스트를 숫자로 바꾸고

LSTM을 통해 의미를 이해하고

1~5 중 어떤 유형의 스팸인지 예측합니다! 🚀